# Set up

In [1]:
import dash
from dash import html, dcc
from dash.dependencies import Output, Input

import geopandas as gpd

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go # to create table using graphical objects

import requests


import os 
import json
from datetime import datetime

import locale

In [2]:
# #*****************************************************************************
# import plotly.io as pio
# # Ensure plots are shown in browser (during exploration)
# pio.renderers.default='browser' # set browser as renderer
# #*****************************************************************************

In [3]:
# show all outputs of cell, not merely of last line (i.e. default of Jupyter Notebook)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Set Belgium time (for Dutch-language indicators of last update time)
locale.setlocale(locale.LC_TIME, 'nl_BE.utf-8')

'nl_BE.utf-8'

# Fetch data

With fetching, there is apparently a limit on the amount of bikes you can fetch (i.e. " Invalid value for limit API parameter: 200 was found but -1 <= limit <= 100 is expected."). 

In [5]:
url_dott = "https://data.stad.gent/api/explore/v2.1/catalog/datasets/dott-deelfietsen-gent/exports/json?lang=nl&timezone=Europe%2FBrussels"

# url_dott = "https://data.stad.gent//api/explore/v2.1/catalog/datasets/dott-deelfietsen-gent/records?limit=20"
# url_dott = "https://data.stad.gent/api/explore/v2.1/catalog/datasets/dott-deelfietsen-gent/records?limit=200"
 

In [6]:
# Fetching data function (when using export function)
def fetch_data_dott_export(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()  
        
        # Convert 'last_reported' string to datetime
        for entry in data:
            entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')
        
        # # Convert to JSON
        # json_data = json.dumps(data
        #                        # , indent=2
        #                       )
        
        # Write new data to file
        with open('../data/dott_fetched_data.json', 'w') as json_file:
            json.dump(data, json_file) 
        print("Data (Dott bikes) fetched")
        # return json_data
    
    else:
        print("Failed to fetch data (Dott bikes)")

# Initialize data and app

Fetch data

In [7]:
	### OPTION 1 ###
	# Fetch data file if it is not yet present
# if not os.path.exists("../data/dott_fetched_data.json"):
#     fetch_data_dott()
	### OPTION 2 ###
	# Fetch data again on each time opening webpage
# fetch_data_dott_api(url_dott)

# data_dott_json = fetch_data_dott_export(url_dott)
fetch_data_dott_export(url_dott)

Data (Dott bikes) fetched


Initialize the Dash app

In [8]:
app = dash.Dash(__name__, 
#		url_base_pathname='/visualisaties/parkeergarages-gent/',
		assets_folder='assets') # Relative path to the folder of css file)
app.title = "Beschikbaarheid Dott fietsen"

Perform initial read-in of the filtered JSON file

In [9]:
with open('../data/dott_fetched_data.json', 'r') as json_file:
    data_dott_json = json.load(json_file)

In [10]:
# # Convert 'last_reported' string to datetime
# for entry in data_dott:
#     entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')

# Transform to dataframe for easier handling
df_dott = pd.DataFrame(data_dott_json)

In [34]:
# # Inspect dataframe
# df_dott.head()
# df_dott.tail()
# df_dott.columns
# df_dott.shape

In [35]:
# data_dott_json

In [13]:
# df_dott.columns

In [14]:
# df_dott["vehicle_type_id"].values

# Function to update graph

In [29]:
def graph_dott(df):
    # Create a hovertemplate
    # hover_template = "<b>%{hovertext}</b><br>" + \
    #                  "Fiets ID: %{customdata[0]}<br>" + \
    #                  "Beschikbare afstand: %{customdata10]}"

    # Replace 'distance' with the actual column name containing the distances in meters
    df['range_km'] = df['current_range_meters'] / 1000
    # Create a new column in the DataFrame with formatted distances
    df['formatted_range'] = df['range_km'].apply(lambda x: f'{x:.1f} km')

    
    hover_template = "<b>Dott</b><br>" + \
                     "Beschikbare afstand: %{customdata[0]}"
    
    # Create a map figure using Plotly Express
    fig = px.scatter_mapbox(
        df,
        lat=df['lat'],
        lon=df['lon'],
        # color=df["vehicle_type_id"],
        # size="current_range_meters",
        # hover_name="bike_id",
        hover_data={"bike_id": True, 
                    "current_range_meters": True
                   },
        # # Create predefined color scale to ensure sufficient contrast, not coming close to white
        # color_continuous_scale=[
        #     [0.0, "red"],
        #     [0.3, "orange"],
        #     [0.5, "yellow"],
        #     [0.7, "lime"],
        #     [1.0, "green"]
        # ],
        # color_continuous_scale="RdYlGn",  # Red (low available capacity) to Green (high available capacity)
        # range_color=[df["is_reserved"].min(), df["is_reserved"].max()],
        # color_discrete_map={"dott_bicycle": "red", "0": "green"},
        # For mapbox_styles, see https://plotly.com/python/mapbox-layers/ 
        mapbox_style="carto-positron", # light
        # mapbox_style="carto-darkmatter", # dark
        # mapbox_style="open-street-map", # street style
        zoom=12,
        # labels={'availablecapacity': 'Beschikbare parkeerplaatsen'}  # Set the legend label
    )
    
    # Update hovertemplate
    fig.update_traces(
        hovertemplate=hover_template
    )
    
    # Set the custom data for hovertemplate
    fig.update_traces(customdata=df[[
        # 'bike_id', 
        'formatted_range'
    ]].values)
    
    
    # Update the layout to hide the color scale legend (shows bad on mobile site)
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
		coloraxis_showscale=False,
        )
    
    return fig


In [30]:
dott_graph = graph_dott(df_dott)

In [31]:
# # *****************************************************************************
# # Show map
# dott_graph.show()
# # *****************************************************************************

# Dash app

## Layout

In [32]:
# Define the app layout, using CSS Bootstrap
app.layout = html.Div([
    html.Link(rel='stylesheet', href='assets/styles.css'),  # Your custom CSS link
    html.Link(rel='stylesheet', href='https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'),  # Bootstrap CSS link

    html.Div(className='container mt-4', children=[
        html.H1(className='text-center', children="Beschikbaarheid dott fietsen"),

        # html.P(className='text-center', children="Beschikbaarheid van de verschillende parkeergarages binnen het Gentse stadscentrum."),

        # html.Div(className='d-flex justify-content-between align-items-center flex-wrap', children=[ # Make button and update indicator more compact
        #     html.Div(id='last-update-time', className='text-center pt-3 pb-2', children=get_last_update_time('../data/last_update.txt')),
        
        #     html.Div(className='text-center', children=[
        #         html.Button("Update", id="refresh-btn", className='btn btn-primary mt-3 mb-3'),
        #         dcc.Interval(id='refresh-interval-component', interval=10*60*1000, n_intervals=0)
        #     ]),
        # ]),
                        
        # html.Div(className='graph-container custom-graph-container', children=[ # Add custom-graph-container next to standard Bootstrap CSS style
            
        #     # Dropdown for selecting display option
        #     dcc.Dropdown(
        #         id='display-option',
        #         options=[
        #             {'label': 'Parkeergarages', 'value': 'parkings'},
        #             {'label': 'Parkeertariefzones', 'value': 'parking-zones'},
        #             {'label': 'Parkeergarages en parkeertariefzones', 'value': 'parkings_AND_parking-zones'},
        #         ],
        #         value='parkings',  # Set default value
        #         multi=False  # Allow only one option to be selected
        #     ),
        #     # Graph                                                                 
        #     dcc.Graph(id='live-update-graph', figure=update_parkings()),
        #     dcc.Interval(id='update-graph-interval', interval=1*1000, n_intervals=0),
        # ]),        

        html.Div(
            dcc.Graph(figure=dott_graph)
        ),
        # html.Footer(className='text-center', children=html.P([
        #     "De gegevens zijn beschikbaar via ",
        #     html.A("Stad Gent API", href="https://data.stad.gent/explore/dataset/bezetting-parkeergarages-real-time/table/?sort=-occupation"),
        #     ". De onderliggende code is beschikbaar op ",
        #     html.A("GitHub", href="https://github.com/NT131/parkeergarages_gent"),
            # "."
        # ]))
    ])
])

## Run app

In [33]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
File ~/software/miniconda3/envs/data_handling_env/lib/python3.11/site-packages/flask/app.py:867, in Flask.full_dispatch_request(self=<Flask '__main__'>)
    865     rv = self.preprocess_request()
    866     if rv is None:
--> 867         rv = self.dispatch_request()
        self = <Flask '__main__'>
        rv = None
    868 except Exception as e:
    869     rv = self.handle_user_exception(e)

File ~/software/miniconda3/envs/data_handling_env/lib/python3.11/site-packages/flask/app.py:852, in Flask.dispatch_request(self=<Flask '__main__'>)
    850 # otherwise dispatch to the handler for that endpoint
    851 view_args: dict[str, t.Any] = req.view_args  # type: ignore[assignment]
--> 852 return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
        rule = <Rule '/assets/<filename>' (HEAD, GET, OPTIONS) -> _dash_assets

# Temp

In [ ]:
# # Fetching data function (when using API)
# def fetch_data_dott_api(url):
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()    
#         # Filter out the row with name "Loop"
#         # filtered_data = [record for record in data.get("results", []) if record.get("name") != "The Loop"]
#         data = [record for record in data.get("results", [])]
        
#         # Write new data to file
#         with open('../data/dott_fetched_data.json', 'w') as json_file:
#             json.dump(data, json_file) 
#         print("Data (Dott bikes) fetched")
    
#     else:
#         print("Failed to fetch data (Dott bikes)")

In [ ]:
# response = requests.get(url_dott)
# data = response.json() 

In [ ]:
# data

In [ ]:
# # Convert 'last_reported' string to datetime
# for entry in data:
#     entry['last_reported'] = datetime.utcfromtimestamp(int(entry['last_reported'])).strftime('%Y-%m-%d %H:%M:%S')

# # Create DataFrame
# df = pd.DataFrame(data)
# df

In [ ]:
# data_dott_json